In [40]:
import pandas as pd
import joblib
import numpy as np
import os

# Set model directory
MODEL_DIR = "../models"

# Load models and scalers
try:
    models = {
        'flood_risk': joblib.load(os.path.join(MODEL_DIR, "xgb_flood_risk_model.pkl")),
        'area': joblib.load(os.path.join(MODEL_DIR, "xgb_area_model.pkl")),
        'population': joblib.load(os.path.join(MODEL_DIR, "xgb_population_model.pkl")),
        'crops': joblib.load(os.path.join(MODEL_DIR, "xgb_crops_model.pkl")),
        'houses': joblib.load(os.path.join(MODEL_DIR, "xgb_houses_model.pkl"))
    }

    scalers = {
        'area': joblib.load(os.path.join(MODEL_DIR, "scaler_area.pkl")),
        'population': joblib.load(os.path.join(MODEL_DIR, "scaler_population.pkl")),
        'crops': joblib.load(os.path.join(MODEL_DIR, "scaler_crops.pkl")),
        'houses': joblib.load(os.path.join(MODEL_DIR, "scaler_houses.pkl"))
    }

except FileNotFoundError as e:
    print(f"❌ Error loading model or scaler files: {e}")
    exit()

# Sample input
data_input = {
    'Year-Month': 202201,  # Dummy value, kept to satisfy model
    'District': 5,
    'Rainfall (mm)': 0.731480275,
    'River': 2,
    'River Level': 2.358254062
}

# Add engineered features
data_input['rainfall_river'] = data_input['Rainfall (mm)'] * data_input['River Level']
data_input['rainfall_district'] = data_input['Rainfall (mm)'] * data_input['District']
data_input['riverlevel_district'] = data_input['River Level'] * data_input['District']

# Feature order (must match training)
X_new = pd.DataFrame([data_input])[[
    'Year-Month', 'District', 'Rainfall (mm)', 'River', 'River Level',
    'rainfall_river', 'rainfall_district', 'riverlevel_district'
]]

# Safe inverse function assuming log1p was used
def safe_inverse_log1p(y_scaled):
    return np.expm1(y_scaled)

# Make predictions
try:
    flood_risk_pred = models['flood_risk'].predict(X_new)[0]

    area_scaled = models['area'].predict(X_new)
    pop_scaled = models['population'].predict(X_new)
    crops_scaled = models['crops'].predict(X_new)
    houses_scaled = models['houses'].predict(X_new)

    print("\n📉 Scaled Outputs:")
    print(f"Area (scaled): {area_scaled[0]:.4f}")
    print(f"Population (scaled): {pop_scaled[0]:.4f}")
    print(f"Crops (scaled): {crops_scaled[0]:.4f}")
    print(f"Houses (scaled): {houses_scaled[0]:.4f}")

    # Inverse transform
    area_original = float(safe_inverse_log1p(area_scaled[0]))
    population_original = float(safe_inverse_log1p(pop_scaled[0]))
    crops_original = float(safe_inverse_log1p(crops_scaled[0]))
    houses_original = float(safe_inverse_log1p(houses_scaled[0]))

    predictions = {
        'Flood Risk': int(flood_risk_pred),
        'Area affected (m.ha)': area_original,
        'Population affected (million)': population_original,
        'Damage to Crops': crops_original,
        'Damage to Houses': houses_original
    }

    print("\n✅ Final Prediction Results:")
    print(f"Flood Risk: {predictions['Flood Risk']} (0=No, 1=Yes, 2=Severe)")
    print(f"Area affected (m.ha): {predictions['Area affected (m.ha)']:.4f}")
    print(f"Population affected (million): {predictions['Population affected (million)']:.4f}")
    print(f"Damage to Crops: {predictions['Damage to Crops']:.4f}")
    print(f"Damage to Houses: {predictions['Damage to Houses']:.4f}")

except Exception as e:
    print(f"❌ Error during prediction: {e}")



📉 Scaled Outputs:
Area (scaled): 0.2718
Population (scaled): 0.4292
Crops (scaled): 0.9754
Houses (scaled): 5.8632

✅ Final Prediction Results:
Flood Risk: 0 (0=No, 1=Yes, 2=Severe)
Area affected (m.ha): 0.3123
Population affected (million): 0.5360
Damage to Crops: 1.6523
Damage to Houses: 350.8307
